In [141]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from getpass import getpass
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

In [142]:
# Connection parameters to PostgreSQL
DB_USER = "postgres"         
DB_PASSWORD = getpass()
DB_HOST = "localhost"         
DB_PORT = "5432"             
DB_NAME = "ecommerce"         

# Create connection
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

 ········


## We want to improve prediction of order delivery rate to increase customers' review scores

## Hence our primary key is order_id

### First let's review our tables

In [6]:
tables_list = ['customers', 'sellers', 'products', 'orders', 'order_items', 'order_payments',
                 'order_reviews', 'geolocation', 'product_category_name_translation']
for t in tables_list:
    print(f"Table {t}")
    print(pd.read_sql(f'''
    SELECT *
    FROM {t}
    limit 5;
    ''', engine))
    print(100*'=' + '\n\n')

Table customers
                            customer_id                    customer_unique_id  \
0  06b8999e-2fba-1a1f-bc88-172c00ba8bc7  861eff47-11a5-42e4-b938-43c6dd7febb0   
1  18955e83-d337-fd6b-2def-6b18a428ac77  290c77bc-529b-7ac9-35b9-3aa66c333dc3   
2  4e7b3e00-2885-86eb-d087-12fdd0374a03  060e732b-5b29-e818-1a18-229c7b0b2b5e   
3  b2b6027b-c5c5-109e-529d-4dc6358b12c3  259dac75-7896-d24d-7702-b9acbbff3f3c   
4  4f2d8ab1-71c8-0ec8-364f-7c12e35b23ad  345ecd01-c38d-18a9-036e-d96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  


Table sellers
                              seller_id  seller_zip_code_prefix  \
0  344

### We can divide features into several categories:
#### 1. Order features (number of products, products features, payments features, distance between seller and customer etc)
#### 2. Seller's features (history of deliveries from this seller, reviews features etc)
#### 3. Customer's features (history of deliveries to this customer etc)

# Customer features

In [66]:
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS customer_features_raw;'))
    connection.commit()
    connection.execute(text('''
    CREATE TABLE customer_features_raw AS
    WITH geo AS 
    (
    SELECT geolocation_zip_code_prefix::varchar || geolocation_city || geolocation_state as address, 
           AVG(geolocation_lat) AS geolocation_lat, 
           AVG(geolocation_lng) AS geolocation_lng
    FROM geolocation
    GROUP BY geolocation_zip_code_prefix::varchar || geolocation_city || geolocation_state
    )
    SELECT c.customer_unique_id as cid
    , c.customer_id, c.customer_zip_code_prefix, c.customer_city, c.customer_state
    /*, o.order_id, o.order_status, o.order_purchase_timestamp, o.order_approved_at, o.order_delivered_carrier_date, o.order_delivered_customer_date, o.order_estimated_delivery_date 
    , p.*
    , oi.seller_id, oi.order_item_id, oi.price, oi.freight_value, oi.shipping_limit_date
    , op.payment_sequential, op.payment_type, op.payment_installments, op.payment_value
    , orev.review_score, orev.review_creation_date, orev.review_answer_timestamp */
    , gc.geolocation_lat as customer_lat, gc.geolocation_lng as customer_lng
    FROM customers c
    /*LEFT JOIN orders o
    ON o.customer_id = c.customer_id
    LEFT JOIN order_items oi 
    ON oi.order_id = o.order_id
    LEFT JOIN products p
    ON p.product_id = oi.product_id
    LEFT JOIN sellers s
    ON s.seller_id = oi.seller_id
    LEFT JOIN order_payments op
    ON op.order_id = o.order_id
    LEFT JOIN order_reviews orev
    ON orev.order_id = o.order_id */
    LEFT JOIN geo gc
    ON gc.address = c.customer_zip_code_prefix::varchar || c.customer_city || c.customer_state
    '''))
    connection.commit()  

In [67]:
pd.read_sql('''
SELECT *
FROM customer_features_raw
LIMIT 5;
''', engine)

,cid,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,861eff47-11a5-42e4-b938-43c6dd7febb0,06b8999e-2fba-1a1f-bc88-172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929
1,290c77bc-529b-7ac9-35b9-3aa66c333dc3,18955e83-d337-fd6b-2def-6b18a428ac77,9790,sao bernardo do campo,SP,-23.728035,-46.542818
2,345ecd01-c38d-18a9-036e-d96c73b8d066,4f2d8ab1-71c8-0ec8-364f-7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925
3,1175e95f-b47d-dff9-de6b-2b06188f7e0d,5adf08e3-4b2e-9939-82a4-7070956c5c65,81560,curitiba,PR,-25.486784,-49.217558
4,9afe194f-b833-f79e-300e-37e580171f22,4b7139f3-4592-b3a3-1687-243a302fa75b,30575,belo horizonte,MG,-19.973801,-43.972640


In [68]:
pd.read_sql('''
SELECT COUNT (*) as rows_cnt
, COUNT (DISTINCT cid) as customer_cnt
--, COUNT (DISTINCT order_id) as order_id_cnt
FROM customer_features_raw
LIMIT 10;
''', engine)

,rows_cnt,customer_cnt
0,99441,96096


### Since for every customer we have approximately one order, there's not sense in considering customer's history. Instead, we'll do some standart feature extraction

# Seller features

In [3]:
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS seller_features_raw;'))
    connection.commit()
    connection.execute(text('''
    CREATE TABLE seller_features_raw AS
    WITH geo AS 
    (
    SELECT geolocation_zip_code_prefix::varchar || geolocation_city || geolocation_state as address, 
           AVG(geolocation_lat) AS geolocation_lat, 
           AVG(geolocation_lng) AS geolocation_lng
    FROM geolocation
    GROUP BY geolocation_zip_code_prefix::varchar || geolocation_city || geolocation_state
    )
    SELECT s.seller_id as sid
    , s.seller_zip_code_prefix, s.seller_city, s.seller_state
    , gs.geolocation_lat as seller_lat, gs.geolocation_lng as seller_lng
    , AVG(orev.review_score) as avg_review_score
    , MIN(orev.review_score) as min_review_score
    , MAX(orev.review_score) as max_review_score
    , STDDEV_POP(orev.review_score) as std_review_score
    FROM sellers s
    LEFT JOIN order_items oi
    ON oi.seller_id = s.seller_id
    LEFT JOIN order_reviews orev
    ON oi.order_id = orev.order_id
    LEFT JOIN geo gs
    ON gs.address = s.seller_zip_code_prefix::varchar || s.seller_city || s.seller_state
    GROUP BY s.seller_id
    , s.seller_zip_code_prefix, s.seller_city, s.seller_state
    , gs.geolocation_lat, gs.geolocation_lng
    '''))
    connection.commit()  

In [4]:
pd.read_sql('''
SELECT COUNT (*) as rows_cnt
, COUNT (DISTINCT sid) as seller_cnt
--, COUNT (DISTINCT order_id) as order_id_cnt
FROM seller_features_raw
LIMIT 10;
''', engine)

,rows_cnt,seller_cnt
0,3095,3095


In [5]:
pd.read_sql('''
SELECT *
FROM seller_features_raw
LIMIT 5;
''', engine)

,sid,seller_zip_code_prefix,seller_city,seller_state,seller_lat,seller_lng,avg_review_score,min_review_score,max_review_score,std_review_score
0,0015a82c-2db0-00af-6aaa-f3ae2ecb0532,9080,santo andre,SP,-23.640627,-46.541793,3.666667,1,5,1.885618
1,001cca7a-e9ae-17fb-1cae-d9dfb1094831,29156,cariacica,ES,-20.278513,-40.411675,3.902542,1,5,1.439101
2,001e6ad4-69a9-0506-0d95-9994f1b41e4f,24754,sao goncalo,RJ,-22.872332,-43.027057,1.000000,1,1,0.000000
3,002100f7-78ce-b843-1b7a-1020ff7ab48f,14405,franca,SP,-20.528759,-47.411110,3.982143,1,5,1.382171
4,003554e2-dce1-76b5-5553-53e4f3555ac8,74565,goiania,GO,-16.640229,-49.274467,5.000000,5,5,0.000000


# Order features
### delivery_time is our target

In [32]:
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS order_features_raw;'))
    connection.commit()
    connection.execute(text('''
    CREATE TABLE order_features_raw AS
    SELECT o.*
    , p.*
    , oi.seller_id, oi.order_item_id, oi.price, oi.freight_value, oi.shipping_limit_date - o.order_approved_at as shipping_limit_days
    , op.payment_sequential, op.payment_type, op.payment_installments, op.payment_value
    , o.order_delivered_customer_date::date - o.order_approved_at::date as delivery_time 
    FROM orders o
    LEFT JOIN order_items oi
    ON oi.order_id = o.order_id
    LEFT JOIN products p
    ON p.product_id = oi.product_id
    LEFT JOIN order_payments op
    ON op.order_id = o.order_id
    WHERE op.payment_sequential = 1
    '''))
    connection.commit()

In [33]:
pd.read_sql('''
SELECT *
FROM order_features_raw
LIMIT 5;
''', engine)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,product_category_name,...,seller_id,order_item_id,price,freight_value,shipping_limit_days,payment_sequential,payment_type,payment_installments,payment_value,delivery_time
0,00018f77-f2f0-320c-5571-90d7a144bdd3,f6dd3ec0-61db-4e39-8762-9fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,e5f2d52b-8021-89ee-6588-65ca93d83a8f,pet_shop,...,dd7ddc04-e1b6-c2c6-1435-2b383efe2d36,1,239.90,19.93,7 days 00:00:00,1,credit_card,3,259.83,16
1,000229ec-3982-24ef-6ca0-657da4fc703e,6489ae5e-4333-f369-3df5-ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,c777355d-18b7-2b67-abbe-ef9df44fd0fd,moveis_decoracao,...,5b51032e-ddd2-42ad-c84c-38acab88f23d,1,199.00,17.87,4 days 00:00:00,1,credit_card,5,216.87,8
2,00024acb-cdf0-a6da-a1e9-31b038114c75,d4eb9395-c8c0-431e-e92f-ce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,7634da15-2a46-10f1-595e-fa32f14722fc,perfumaria,...,9d7a1d34-a505-2409-0064-25275ba1c2b4,1,12.99,12.79,7 days 00:00:00,1,credit_card,2,25.78,6
3,00042b26-cf59-d7ce-69df-abb4e55b4fd9,58dbd0b2-d702-06bf-40e6-2cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,ac6c3623-068f-30de-0304-5865e4e10089,ferramentas_jardim,...,df560393-f3a5-1e74-553a-b94004ba5c87,1,199.90,18.14,8 days 23:47:38,1,credit_card,3,218.04,25
4,00048cc3-ae77-7c65-dbb7-d2a0634bc1ea,816cbea9-69fe-5b68-9b39-cfc97a506742,delivered,2017-05-15 21:42:34,2017-05-17 03:55:27,2017-05-17 11:05:55,2017-05-22 13:44:35,2017-06-06,ef92defd-e845-ab84-50f9-d70c526ef70f,utilidades_domesticas,...,6426d21a-ca40-2a13-1fc0-a5d0960a3c90,1,21.90,12.69,6 days 00:00:00,1,boleto,1,34.59,5


In [34]:
pd.read_sql('''
SELECT COUNT (*) as rows_cnt
, COUNT (DISTINCT order_id) as order_id_cnt
, COUNT (DISTINCT order_id::varchar || product_id::varchar || order_item_id::varchar) as order_w_product_cnt
FROM order_features_raw
LIMIT 10;
''', engine)

,rows_cnt,order_id_cnt,order_w_product_cnt
0,113332,99360,112558


# Join customers and sellers to orders

In [35]:
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS order_features_raw_w_cust_n_sell;'))
    connection.commit()
    connection.execute(text('''
    CREATE TABLE order_features_raw_w_cust_n_sell AS
    SELECT o.*
    , seller_zip_code_prefix, seller_city, seller_state, seller_lat, seller_lng
    , avg_review_score, min_review_score, max_review_score, std_review_score
    , cid, customer_zip_code_prefix, customer_city, customer_state, customer_lat, customer_lng
    FROM order_features_raw o
    LEFT JOIN customer_features_raw c
    ON c.customer_id = o.customer_id
    LEFT JOIN seller_features_raw s
    ON s.sid = o.seller_id
    '''))
    connection.commit()

In [36]:
pd.read_sql('''
SELECT COUNT (*) as rows_cnt
, COUNT (DISTINCT order_id) as order_id_cnt
, COUNT (DISTINCT order_id::varchar || product_id::varchar || order_item_id::varchar) as order_w_product_cnt
FROM order_features_raw_w_cust_n_sell
LIMIT 10;
''', engine)

,rows_cnt,order_id_cnt,order_w_product_cnt
0,113332,99360,112558


In [37]:
pd.read_sql('''
SELECT *
FROM order_features_raw_w_cust_n_sell
LIMIT 5;
''', engine)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,product_category_name,...,avg_review_score,min_review_score,max_review_score,std_review_score,cid,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,00018f77-f2f0-320c-5571-90d7a144bdd3,f6dd3ec0-61db-4e39-8762-9fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,e5f2d52b-8021-89ee-6588-65ca93d83a8f,pet_shop,...,3.760563,1,5,1.443439,eb28e67c-4c0b-8384-6050-ddfb8a35d051,15775,santa fe do sul,SP,-20.223125,-50.899863
1,000229ec-3982-24ef-6ca0-657da4fc703e,6489ae5e-4333-f369-3df5-ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,c777355d-18b7-2b67-abbe-ef9df44fd0fd,moveis_decoracao,...,3.785714,1,5,1.263700,3818d81c-6709-e39d-06b2-738a8d3a2474,35661,para de minas,MG,-19.870712,-44.593399
2,00024acb-cdf0-a6da-a1e9-31b038114c75,d4eb9395-c8c0-431e-e92f-ce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,7634da15-2a46-10f1-595e-fa32f14722fc,perfumaria,...,3.750000,1,5,1.677051,af861d43-6cfc-08b2-c2dd-efd0ba074622,12952,atibaia,SP,-23.089925,-46.611654
3,00042b26-cf59-d7ce-69df-abb4e55b4fd9,58dbd0b2-d702-06bf-40e6-2cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,ac6c3623-068f-30de-0304-5865e4e10089,ferramentas_jardim,...,3.724138,1,5,1.310345,64b576fb-70d4-41e8-f1b2-d7d446e483c5,13226,varzea paulista,SP,-23.243402,-46.827614
4,00048cc3-ae77-7c65-dbb7-d2a0634bc1ea,816cbea9-69fe-5b68-9b39-cfc97a506742,delivered,2017-05-15 21:42:34,2017-05-17 03:55:27,2017-05-17 11:05:55,2017-05-22 13:44:35,2017-06-06,ef92defd-e845-ab84-50f9-d70c526ef70f,utilidades_domesticas,...,4.173913,1,5,1.048899,85c835d1-28be-ae5b-4ce8-602c491bf385,38017,uberaba,MG,-19.740199,-47.924793


# Load to df and aggregate

In [108]:
df = pd.read_sql('''
SELECT *
FROM order_features_raw_w_cust_n_sell
''', engine)

df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,product_category_name,...,avg_review_score,min_review_score,max_review_score,std_review_score,cid,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,00018f77-f2f0-320c-5571-90d7a144bdd3,f6dd3ec0-61db-4e39-8762-9fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,e5f2d52b-8021-89ee-6588-65ca93d83a8f,pet_shop,...,3.760563,1.0,5.0,1.443439,eb28e67c-4c0b-8384-6050-ddfb8a35d051,15775,santa fe do sul,SP,-20.223125,-50.899863
1,000229ec-3982-24ef-6ca0-657da4fc703e,6489ae5e-4333-f369-3df5-ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,c777355d-18b7-2b67-abbe-ef9df44fd0fd,moveis_decoracao,...,3.785714,1.0,5.0,1.263700,3818d81c-6709-e39d-06b2-738a8d3a2474,35661,para de minas,MG,-19.870712,-44.593399
2,00024acb-cdf0-a6da-a1e9-31b038114c75,d4eb9395-c8c0-431e-e92f-ce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,7634da15-2a46-10f1-595e-fa32f14722fc,perfumaria,...,3.750000,1.0,5.0,1.677051,af861d43-6cfc-08b2-c2dd-efd0ba074622,12952,atibaia,SP,-23.089925,-46.611654
3,00042b26-cf59-d7ce-69df-abb4e55b4fd9,58dbd0b2-d702-06bf-40e6-2cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,ac6c3623-068f-30de-0304-5865e4e10089,ferramentas_jardim,...,3.724138,1.0,5.0,1.310345,64b576fb-70d4-41e8-f1b2-d7d446e483c5,13226,varzea paulista,SP,-23.243402,-46.827614
4,00048cc3-ae77-7c65-dbb7-d2a0634bc1ea,816cbea9-69fe-5b68-9b39-cfc97a506742,delivered,2017-05-15 21:42:34,2017-05-17 03:55:27,2017-05-17 11:05:55,2017-05-22 13:44:35,2017-06-06,ef92defd-e845-ab84-50f9-d70c526ef70f,utilidades_domesticas,...,4.173913,1.0,5.0,1.048899,85c835d1-28be-ae5b-4ce8-602c491bf385,38017,uberaba,MG,-19.740199,-47.924793


In [109]:
df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_id', 'order_item_id', 'price', 'freight_value',
       'shipping_limit_days', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'delivery_time',
       'seller_zip_code_prefix', 'seller_city', 'seller_state', 'seller_lat',
       'seller_lng', 'avg_review_score', 'min_review_score',
       'max_review_score', 'std_review_score', 'cid',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'customer_lat', 'customer_lng'],
      dtype='object')

In [110]:
# # Features those need aggregation on order_id level (without categorical product_category_name)
agg_features = [
    "product_name_lenght", "product_description_lenght", "product_photos_qty", 
    "product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm",
    "price", "freight_value", "shipping_limit_days"
]

# Features corrspondin to order_id as 1 to 1 (we agregate them with max)
keep_features = [
    "order_purchase_timestamp", "order_approved_at", "order_estimated_delivery_date",
    "payment_sequential", "payment_type", "payment_installments", "payment_value",
    "seller_zip_code_prefix", "seller_city", "seller_state", "seller_lat", "seller_lng", 
    "avg_review_score", "min_review_score", "max_review_score", "std_review_score", 
    "customer_zip_code_prefix", "customer_city", "customer_state", "customer_lat", "customer_lng", 
    "delivery_time"
]

# Agg functions for agg_features
agg_funcs = {col: ["min", "max", "mean", "median", "std"] for col in agg_features}

# Take top 10 values of product_category_name. Others go to other category
top_10_categories = df["product_category_name"].value_counts().index[:10].tolist()
df["product_category_name"] = df["product_category_name"].apply(lambda x: x if x in top_10_categories else "other_category")

# One-hot encoding 
df_cat = pd.get_dummies(df["product_category_name"], prefix="category_count")
df = pd.concat([df, df_cat], axis=1)

# Add categories to aggregation functions list (we apply sum as agg func to get total amount of each category within an order)
category_cols = df_cat.columns.tolist()
for cat in category_cols:
    agg_funcs[cat] = "sum"

# Group by order id
df_agg = df.groupby("order_id").agg(agg_funcs).reset_index()
df_agg.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in df_agg.columns]
df_agg = df_agg.rename(columns={"order_id_": "order_id"})

# Calculate rate of top (within an order) category in its order
category_cols = [col + "_sum" for col in category_cols]  
category_share_cols = [col.replace("category_count", "category_share") for col in category_cols]
df_agg[category_share_cols] = df_agg[category_cols].div(df_agg[category_cols].sum(axis=1), axis=0)

# Add keep_features
df_keep = df.groupby("order_id")[keep_features].max().reset_index()

# Merge 2 dfs
df_final = df_agg.merge(df_keep, on="order_id", how="left")

df_final.head()

,order_id,product_name_lenght_min,product_name_lenght_max,product_name_lenght_mean,product_name_lenght_median,product_name_lenght_std,product_description_lenght_min,product_description_lenght_max,product_description_lenght_mean,product_description_lenght_median,...,avg_review_score,min_review_score,max_review_score,std_review_score,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,delivery_time
0,00010242-fe8c-5a6d-1ba2-dd792cb16214,58.0,58.0,58.0,58.0,NaN,598.0,598.0,598.0,598.0,...,4.046358,1.0,5.0,1.338863,28013,campos dos goytacazes,RJ,-21.762775,-41.309633,7.0
1,00018f77-f2f0-320c-5571-90d7a144bdd3,56.0,56.0,56.0,56.0,NaN,239.0,239.0,239.0,239.0,...,3.760563,1.0,5.0,1.443439,15775,santa fe do sul,SP,-20.223125,-50.899863,16.0
2,000229ec-3982-24ef-6ca0-657da4fc703e,59.0,59.0,59.0,59.0,NaN,695.0,695.0,695.0,695.0,...,3.785714,1.0,5.0,1.263700,35661,para de minas,MG,-19.870712,-44.593399,8.0
3,00024acb-cdf0-a6da-a1e9-31b038114c75,42.0,42.0,42.0,42.0,NaN,480.0,480.0,480.0,480.0,...,3.750000,1.0,5.0,1.677051,12952,atibaia,SP,-23.089925,-46.611654,6.0
4,00042b26-cf59-d7ce-69df-abb4e55b4fd9,59.0,59.0,59.0,59.0,NaN,409.0,409.0,409.0,409.0,...,3.724138,1.0,5.0,1.310345,13226,varzea paulista,SP,-23.243402,-46.827614,25.0


In [111]:
df_final.columns

Index(['order_id', 'product_name_lenght_min', 'product_name_lenght_max',
       'product_name_lenght_mean', 'product_name_lenght_median',
       'product_name_lenght_std', 'product_description_lenght_min',
       'product_description_lenght_max', 'product_description_lenght_mean',
       'product_description_lenght_median', 'product_description_lenght_std',
       'product_photos_qty_min', 'product_photos_qty_max',
       'product_photos_qty_mean', 'product_photos_qty_median',
       'product_photos_qty_std', 'product_weight_g_min',
       'product_weight_g_max', 'product_weight_g_mean',
       'product_weight_g_median', 'product_weight_g_std',
       'product_length_cm_min', 'product_length_cm_max',
       'product_length_cm_mean', 'product_length_cm_median',
       'product_length_cm_std', 'product_height_cm_min',
       'product_height_cm_max', 'product_height_cm_mean',
       'product_height_cm_median', 'product_height_cm_std',
       'product_width_cm_min', 'product_width_cm_max',

In [112]:
# Create day interval feature from order_estimated_delivery_date and drop unnecessary columns
df_final['estimated_delivery_days'] = df_final.apply(lambda x: (x.order_estimated_delivery_date - x.order_approved_at).days, axis=1)
df_final_trunc = df_final.drop(columns=['seller_zip_code_prefix', 'seller_city',
       'seller_state', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'order_purchase_timestamp',
       'order_approved_at'])
                               

In [113]:
df_final_trunc.columns

Index(['order_id', 'product_name_lenght_min', 'product_name_lenght_max',
       'product_name_lenght_mean', 'product_name_lenght_median',
       'product_name_lenght_std', 'product_description_lenght_min',
       'product_description_lenght_max', 'product_description_lenght_mean',
       'product_description_lenght_median', 'product_description_lenght_std',
       'product_photos_qty_min', 'product_photos_qty_max',
       'product_photos_qty_mean', 'product_photos_qty_median',
       'product_photos_qty_std', 'product_weight_g_min',
       'product_weight_g_max', 'product_weight_g_mean',
       'product_weight_g_median', 'product_weight_g_std',
       'product_length_cm_min', 'product_length_cm_max',
       'product_length_cm_mean', 'product_length_cm_median',
       'product_length_cm_std', 'product_height_cm_min',
       'product_height_cm_max', 'product_height_cm_mean',
       'product_height_cm_median', 'product_height_cm_std',
       'product_width_cm_min', 'product_width_cm_max',

### Encode payment_type into dummies

In [116]:
top_10_categories = df_final_trunc["payment_type"].value_counts().index[:5].tolist()
df_final_trunc["payment_type"] = df_final_trunc["payment_type"].apply(lambda x: x if x in top_10_categories else "other_payment_type")

df_final_trunc_cat = pd.get_dummies(df_final_trunc["payment_type"], prefix="type_count")
df_final_2 = pd.concat([df_final_trunc, df_final_trunc_cat], axis=1)

In [122]:
df_final_2.drop(columns=['payment_type'], inplace=True)
df_final_2.head()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


,order_id,product_name_lenght_min,product_name_lenght_max,product_name_lenght_mean,product_name_lenght_median,product_name_lenght_std,product_description_lenght_min,product_description_lenght_max,product_description_lenght_mean,product_description_lenght_median,...,std_review_score,customer_lat,customer_lng,delivery_time,estimated_delivery_days,type_count_boleto,type_count_credit_card,type_count_debit_card,type_count_not_defined,type_count_voucher
0,00010242-fe8c-5a6d-1ba2-dd792cb16214,58.0,58.0,58.0,58.0,NaN,598.0,598.0,598.0,598.0,...,1.338863,-21.762775,-41.309633,7.0,15.0,False,True,False,False,False
1,00018f77-f2f0-320c-5571-90d7a144bdd3,56.0,56.0,56.0,56.0,NaN,239.0,239.0,239.0,239.0,...,1.443439,-20.223125,-50.899863,16.0,18.0,False,True,False,False,False
2,000229ec-3982-24ef-6ca0-657da4fc703e,59.0,59.0,59.0,59.0,NaN,695.0,695.0,695.0,695.0,...,1.263700,-19.870712,-44.593399,8.0,21.0,False,True,False,False,False
3,00024acb-cdf0-a6da-a1e9-31b038114c75,42.0,42.0,42.0,42.0,NaN,480.0,480.0,480.0,480.0,...,1.677051,-23.089925,-46.611654,6.0,11.0,False,True,False,False,False
4,00042b26-cf59-d7ce-69df-abb4e55b4fd9,59.0,59.0,59.0,59.0,NaN,409.0,409.0,409.0,409.0,...,1.310345,-23.243402,-46.827614,25.0,40.0,False,True,False,False,False


In [123]:
df_final_2.columns

Index(['order_id', 'product_name_lenght_min', 'product_name_lenght_max',
       'product_name_lenght_mean', 'product_name_lenght_median',
       'product_name_lenght_std', 'product_description_lenght_min',
       'product_description_lenght_max', 'product_description_lenght_mean',
       'product_description_lenght_median', 'product_description_lenght_std',
       'product_photos_qty_min', 'product_photos_qty_max',
       'product_photos_qty_mean', 'product_photos_qty_median',
       'product_photos_qty_std', 'product_weight_g_min',
       'product_weight_g_max', 'product_weight_g_mean',
       'product_weight_g_median', 'product_weight_g_std',
       'product_length_cm_min', 'product_length_cm_max',
       'product_length_cm_mean', 'product_length_cm_median',
       'product_length_cm_std', 'product_height_cm_min',
       'product_height_cm_max', 'product_height_cm_mean',
       'product_height_cm_median', 'product_height_cm_std',
       'product_width_cm_min', 'product_width_cm_max',

### Calculate distance between seller and customer

In [125]:
df_final_2['sel2cust_dist'] = df_final_2.apply(lambda x: math.sqrt((x.seller_lat - x.customer_lat)**2 + (x.seller_lng - x.customer_lng)**2), axis=1)
df_final_2.sel2cust_dist

0              NaN
1         5.510120
2         2.865357
3         2.652272
4         6.316069
           ...    
99355    24.803301
99356     3.351783
99357     3.241389
99358     0.659452
99359     1.222836
Name: sel2cust_dist, Length: 99360, dtype: float64

In [126]:
df_final_3 = df_final_2.drop(columns=['seller_lat', 'seller_lng', 'customer_lat', 'customer_lng'])
df_final_3.head()

,order_id,product_name_lenght_min,product_name_lenght_max,product_name_lenght_mean,product_name_lenght_median,product_name_lenght_std,product_description_lenght_min,product_description_lenght_max,product_description_lenght_mean,product_description_lenght_median,...,max_review_score,std_review_score,delivery_time,estimated_delivery_days,type_count_boleto,type_count_credit_card,type_count_debit_card,type_count_not_defined,type_count_voucher,sel2cust_dist
0,00010242-fe8c-5a6d-1ba2-dd792cb16214,58.0,58.0,58.0,58.0,NaN,598.0,598.0,598.0,598.0,...,5.0,1.338863,7.0,15.0,False,True,False,False,False,NaN
1,00018f77-f2f0-320c-5571-90d7a144bdd3,56.0,56.0,56.0,56.0,NaN,239.0,239.0,239.0,239.0,...,5.0,1.443439,16.0,18.0,False,True,False,False,False,5.510120
2,000229ec-3982-24ef-6ca0-657da4fc703e,59.0,59.0,59.0,59.0,NaN,695.0,695.0,695.0,695.0,...,5.0,1.263700,8.0,21.0,False,True,False,False,False,2.865357
3,00024acb-cdf0-a6da-a1e9-31b038114c75,42.0,42.0,42.0,42.0,NaN,480.0,480.0,480.0,480.0,...,5.0,1.677051,6.0,11.0,False,True,False,False,False,2.652272
4,00042b26-cf59-d7ce-69df-abb4e55b4fd9,59.0,59.0,59.0,59.0,NaN,409.0,409.0,409.0,409.0,...,5.0,1.310345,25.0,40.0,False,True,False,False,False,6.316069


### Deal with NaN in dataframe

In [132]:
for c in df_final_3.columns:
    s = np.sum(df_final_3[c].isna())
    if s > 0:
        print(s, c)

2160 product_name_lenght_min
2160 product_name_lenght_max
2160 product_name_lenght_mean
2160 product_name_lenght_median
89712 product_name_lenght_std
2160 product_description_lenght_min
2160 product_description_lenght_max
2160 product_description_lenght_mean
2160 product_description_lenght_median
89712 product_description_lenght_std
2160 product_photos_qty_min
2160 product_photos_qty_max
2160 product_photos_qty_mean
2160 product_photos_qty_median
89712 product_photos_qty_std
790 product_weight_g_min
790 product_weight_g_max
790 product_weight_g_mean
790 product_weight_g_median
89566 product_weight_g_std
790 product_length_cm_min
790 product_length_cm_max
790 product_length_cm_mean
790 product_length_cm_median
89566 product_length_cm_std
790 product_height_cm_min
790 product_height_cm_max
790 product_height_cm_mean
790 product_height_cm_median
89566 product_height_cm_std
790 product_width_cm_min
790 product_width_cm_max
790 product_width_cm_mean
790 product_width_cm_median
89566 product

In [133]:
df_final_4 = df_final_3[~df_final_3.delivery_time.isna()].fillna(0)
df_final_4.head()

,order_id,product_name_lenght_min,product_name_lenght_max,product_name_lenght_mean,product_name_lenght_median,product_name_lenght_std,product_description_lenght_min,product_description_lenght_max,product_description_lenght_mean,product_description_lenght_median,...,max_review_score,std_review_score,delivery_time,estimated_delivery_days,type_count_boleto,type_count_credit_card,type_count_debit_card,type_count_not_defined,type_count_voucher,sel2cust_dist
0,00010242-fe8c-5a6d-1ba2-dd792cb16214,58.0,58.0,58.0,58.0,0.0,598.0,598.0,598.0,598.0,...,5.0,1.338863,7.0,15.0,False,True,False,False,False,0.000000
1,00018f77-f2f0-320c-5571-90d7a144bdd3,56.0,56.0,56.0,56.0,0.0,239.0,239.0,239.0,239.0,...,5.0,1.443439,16.0,18.0,False,True,False,False,False,5.510120
2,000229ec-3982-24ef-6ca0-657da4fc703e,59.0,59.0,59.0,59.0,0.0,695.0,695.0,695.0,695.0,...,5.0,1.263700,8.0,21.0,False,True,False,False,False,2.865357
3,00024acb-cdf0-a6da-a1e9-31b038114c75,42.0,42.0,42.0,42.0,0.0,480.0,480.0,480.0,480.0,...,5.0,1.677051,6.0,11.0,False,True,False,False,False,2.652272
4,00042b26-cf59-d7ce-69df-abb4e55b4fd9,59.0,59.0,59.0,59.0,0.0,409.0,409.0,409.0,409.0,...,5.0,1.310345,25.0,40.0,False,True,False,False,False,6.316069


In [134]:
for c in df_final_4.columns:
    s = np.sum(df_final_4[c].isna())
    if s > 0:
        print(s, c)

### Load final dataframe to postgres and make sure everything's fine

In [135]:
df_final_4.to_sql('order_sample_ready_for_ml', engine, if_exists="append", index=False)

/var/folders/q8/ftp_43616hxb8y78vh3691gw0000gn/T/ipykernel_5237/2515938837.py:1: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  df_final_4.to_sql('order_sample_ready_for_ml', engine, if_exists="append", index=False)


229

In [136]:
pd.read_sql('''
SELECT *
FROM order_sample_ready_for_ml
LIMIT 5;
''', engine)

,order_id,product_name_lenght_min,product_name_lenght_max,product_name_lenght_mean,product_name_lenght_median,product_name_lenght_std,product_description_lenght_min,product_description_lenght_max,product_description_lenght_mean,product_description_lenght_median,...,max_review_score,std_review_score,delivery_time,estimated_delivery_days,type_count_boleto,type_count_credit_card,type_count_debit_card,type_count_not_defined,type_count_voucher,sel2cust_dist
0,00010242-fe8c-5a6d-1ba2-dd792cb16214,58.0,58.0,58.0,58.0,0.0,598.0,598.0,598.0,598.0,...,5.0,1.338863,7.0,15.0,False,True,False,False,False,0.000000
1,00018f77-f2f0-320c-5571-90d7a144bdd3,56.0,56.0,56.0,56.0,0.0,239.0,239.0,239.0,239.0,...,5.0,1.443439,16.0,18.0,False,True,False,False,False,5.510120
2,000229ec-3982-24ef-6ca0-657da4fc703e,59.0,59.0,59.0,59.0,0.0,695.0,695.0,695.0,695.0,...,5.0,1.263700,8.0,21.0,False,True,False,False,False,2.865357
3,00024acb-cdf0-a6da-a1e9-31b038114c75,42.0,42.0,42.0,42.0,0.0,480.0,480.0,480.0,480.0,...,5.0,1.677051,6.0,11.0,False,True,False,False,False,2.652272
4,00042b26-cf59-d7ce-69df-abb4e55b4fd9,59.0,59.0,59.0,59.0,0.0,409.0,409.0,409.0,409.0,...,5.0,1.310345,25.0,40.0,False,True,False,False,False,6.316069


In [137]:
pd.read_sql('''
SELECT COUNT(*)
FROM order_sample_ready_for_ml
LIMIT 5;
''', engine)

,count
0,96383


In [138]:
df_final_4.shape

(96383, 89)

# Sample order_sample_ready_for_ml is ready to train ML model